In [1]:
from enum import Enum
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

import pandas as pd
from datetime import datetime, date, timedelta
import numpy as np
import io
import sys
import re
import os

sys.path.insert(0,r'C:\Users\stesl0131\Desktop\jupyter notebook dir\connection')
from connection import connections as c

## Functions

In [2]:
def export_excel(df):
  with io.BytesIO() as buffer:
    writer = pd.ExcelWriter(buffer)
    df.to_excel(writer)
    writer.save()
    return buffer.getvalue()

def export_csv(df):
  with io.StringIO() as buffer:
    df.to_csv(buffer)
    return buffer.getvalue()

## All DF ops

In [ ]:
redshift_datateam_con=c.redshiftcon_datateam()

query='''SELECT 
TO_CHAR(t1.first_inst_date,'yyyy-mm') as first_inst_month
--,case when  ea.successfull_registration=1 then 'enach registered' else 'enach not registered'  end   as enach
--,case when na.nachaccepted =1 then 'nach_registered' else 'nach_not_registered' end as p_nach 
,case when na.nachaccepted =1 then  'Yes' 
      when ea.successfull_registration=1  then 'Yes' 
      else 'No' end as nach_registered
 ,sum(t1.eligible_first_pd7) as total_elegible_fpd7
,sum(t1.first_pd7) as fpd7
,sum(t1.eligible_first_pd15) as total_elegible_fpd15
,sum(t1.first_pd15) as fpd15
,sum(t1.eligible_first_pd30) as total_elegible_fpd30
,sum(t1.first_pd30) as fpd30
from data_analytics.f_delq_tt t1 
inner join mydbinstance_website_live.loan_tb t2 on t1.loan_id =t2.id
inner join data_analytics.st_prod_elv_product slp on t1.loan_id =slp.id 
left join data_analytics.enach_accepted ea on t1.customer_id =ea.customer_id
left join data_analytics.nach_accepted na on t1.customer_id =na.customer_id 
--where --COALESCE(t2.product_code,'') not like 'prod_elv%' 
where slp.product_type like '%prod_lc%'
--COALESCE(t2.product_code,'')  like 'prod_elv%' 
group by 1,2 limit 100;'''

prod_lc = pd.read_sql(query , redshift_datateam_con)



redshift_datateam_con=c.redshiftcon_datateam()

query='''SELECT 
TO_CHAR(t1.first_inst_date,'yyyy-mm') as first_inst_month
--,case when  ea.successfull_registration=1 then 'enach registered' else 'enach not registered'  end   as enach
--,case when na.nachaccepted =1 then 'nach_registered' else 'nach_not_registered' end as p_nach 
,case when na.nachaccepted =1 then  'Yes' 
      when ea.successfull_registration=1  then 'Yes' 
      else 'No' end as nach_registered
 ,sum(t1.eligible_first_pd7) as total_elegible_fpd7
,sum(t1.first_pd7) as fpd7
,sum(t1.eligible_first_pd15) as total_elegible_fpd15
,sum(t1.first_pd15) as fpd15
,sum(t1.eligible_first_pd30) as total_elegible_fpd30
from data_analytics.f_delq_tt t1 
inner join mydbinstance_website_live.loan_tb t2 on t1.loan_id =t2.id
inner join data_analytics.st_libr8_product slp on t1.loan_id =slp.id 
left join data_analytics.enach_accepted ea on t1.customer_id =ea.customer_id
left join data_analytics.nach_accepted na on t1.customer_id =na.customer_id 
--where --COALESCE(t2.product_code,'') not like 'prod_elv%' 
where --slp.product_type not like '%prod_lc%'
COALESCE(t2.product_code,'')  like 'prod_elv%' 
group by 1,2 limit 10;'''

prod_lc1 = pd.read_sql(query , redshift_datateam_con)

## List of dataframe

In [20]:
### make the data into attachable .csv or .xlsx format
a=export_csv(LOC1)
b=export_excel(LOC)

EXPORTERS = {'LOC.csv': a, 'dataframe.xlsx': b}

## Email Recipients

In [22]:
recipients = ['recipients1@gmail.com','recipients2@gmail.com']
cc=["'recipients1@gmail.com'"]
sender = "sender@gmail.com"
password='senderpassword'
subject = "risk"

## Define mail Subject

In [23]:
subject = "This is a test email"

In [24]:
message=MIMEMultipart('alternative')
message['subject']=subject
message['to']=",".join(recipients)
message['cc']=",".join(cc)
message['from']=sender

## Write body of the mail

In [25]:
body=html = """\
<html>
  <body>
    <p>Greeting to all,<br>
       please find the data for  the  LOC <br>
       {0}
       <br><br><br>
       
       please find the data for elev8 <br>
       {1}
       <br>
    </p>
    <p><b> This is a auto email </b></P>
    <br><br>
    <P>Thanks and Regards</p>
    <p>Data Team</P>
  </body>
</html>
""".format(df1.to_html(), df2.to_html())

In [26]:
b=MIMEText(body, "html")
message.attach(b)

for filename in EXPORTERS:    
    attachment = MIMEApplication(EXPORTERS[filename])
    attachment['Content-Disposition'] = 'attachment; filename="{}"'.format(filename)
    message.attach(attachment)

## Send Mail

In [ ]:
session = smtplib.SMTP('smtp.gmail.com', 587)
session.starttls()
session.login(sender, password)
send_it = session.sendmail(sender, recipients, message.as_string())
session.quit()